# Notebook 1: Load_and_Save_Raw_Therapy

## Objetivo
Carregar os dados de todos os 5 trimestres da tabela `Therapy` e salvar em formato Parquet.

## Passos

1. **Carregar dados de todos os trimestres**
    - Vamos carregar os dados de `Therapy` para os trimestres 2022Q4, 2023Q1, 2023Q2, 2023Q3 e 2023Q4. 
    - Spark vai inferir automaticamente o header e o schema, mas somente no próximo notebook será definido manualmente. 
    
2. **Criar DataFrames**
    - Cada trimestre será carregado em um DataFrame separado.
    
3. **Concatenar DataFrames**
    - Todos os DataFrames serão concatenados num único DataFrame.
    
4. **Salvar em formato Parquet**
    - O DataFrame concatenado será salvo em formato Parquet para uso eficiente em análises futuras.



In [0]:
trimestres = ['2022Q4', '2023q1', '2023q2', '2023Q3', '2023Q4']
dfs_therapy = []



In [0]:
for trimestre in trimestres:
    ano = trimestre[:4]
    quarter = trimestre[-1]
    path = f'/FileStore/faers-downloaded/faers_ascii_{trimestre}/ASCII/THER{ano[-2:]}Q{quarter}.txt'
    try:
        df = spark.read.csv(path, sep='$', header=True, inferSchema=True)
        dfs_therapy.append(df)
        print(f"Arquivo carregado com sucesso: {path}")
    except Exception as e:
        print(f"Erro ao carregar {path}: {str(e)}")



Arquivo carregado com sucesso: /FileStore/faers-downloaded/faers_ascii_2022Q4/ASCII/THER22Q4.txt
Arquivo carregado com sucesso: /FileStore/faers-downloaded/faers_ascii_2023q1/ASCII/THER23Q1.txt
Arquivo carregado com sucesso: /FileStore/faers-downloaded/faers_ascii_2023q2/ASCII/THER23Q2.txt
Arquivo carregado com sucesso: /FileStore/faers-downloaded/faers_ascii_2023Q3/ASCII/THER23Q3.txt
Arquivo carregado com sucesso: /FileStore/faers-downloaded/faers_ascii_2023Q4/ASCII/THER23Q4.txt


In [0]:
#unir dataframes 
if dfs_therapy:
    df_therapy_all = dfs_therapy[0]
    for df in dfs_therapy[1:]:
        df_therapy_all = df_therapy_all.unionAll(df)
    print("Todos os dataframes foram unidos com sucesso.")
    print(f"Número total de linhas: {df_therapy_all.count()}")
else:
    print("Nenhum arquivo foi carregado com sucesso.")

Todos os dataframes foram unidos com sucesso.
Número total de linhas: 3314417


In [0]:
# Mostrar as primeiras linhas e o schema
df_therapy_all.limit(5).display()

primaryid,caseid,dsg_drug_seq,start_dt,end_dt,dur,dur_cod
100115733,10011573,1,20120407,20120409,63,DAY
1002130537,10021305,1,20131219,20131219,1,DAY
1002130537,10021305,4,20140114,null,null,null
1002130537,10021305,5,20191119,null,null,null
100234223,10023422,1,200905,2012,3,YR


In [0]:
#mostrar schema
df_therapy_all.printSchema()

root
 |-- primaryid: long (nullable = true)
 |-- caseid: integer (nullable = true)
 |-- dsg_drug_seq: integer (nullable = true)
 |-- start_dt: integer (nullable = true)
 |-- end_dt: integer (nullable = true)
 |-- dur: integer (nullable = true)
 |-- dur_cod: string (nullable = true)



In [0]:
# modo overwrite 
df_therapy_all.write.mode('overwrite').parquet("dbfs:/FileStore/FAERS-grupo-4/therapy_raw")

In [0]:
# Conclusão
print("Dados de todos os quarters carregados e salvos em formato Parquet.")

Dados de todos os quarters carregados e salvos em formato Parquet.
